In [2]:
import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader



In [3]:
all_pdf_file = glob.glob(r'pdf_data\\*.pdf')
docs = []

for pdf in all_pdf_file:
    loader = PyMuPDFLoader(pdf)
    docs.extend(loader.load())


In [4]:
for i , page in enumerate(docs):
    if i < 2:
        print(page.page_content)
    



DIETARY GUIDELINES 
FOR INDIANS
-A Manual 
NATIONAL INSTITUTE OF NUTRITION
Hyderabad – 500 007, INDIA


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000 , chunk_overlap=10)

In [6]:
chunks = text_splitter.split_documents(docs)
chunks[:5]

[Document(metadata={'producer': 'eDocument Library version 2.6 PDF Filter', 'creator': 'Corel PDF Fusion v1.0', 'creationdate': '2012-05-08T15:10:52+05:30', 'source': 'pdf_data\\DietaryGuidelinesforNINwebsite.pdf', 'file_path': 'pdf_data\\DietaryGuidelinesforNINwebsite.pdf', 'total_pages': 139, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2012-08-13T11:30:02+05:30', 'trapped': '', 'modDate': "D:20120813113002+05'30'", 'creationDate': "D:20120508151052+05'30'", 'page': 1}, page_content='DIETARY GUIDELINES \nFOR INDIANS\n-A Manual \nNATIONAL INSTITUTE OF NUTRITION\nHyderabad – 500 007, INDIA'),
 Document(metadata={'producer': 'eDocument Library version 2.6 PDF Filter', 'creator': 'Corel PDF Fusion v1.0', 'creationdate': '2012-05-08T15:10:52+05:30', 'source': 'pdf_data\\DietaryGuidelinesforNINwebsite.pdf', 'file_path': 'pdf_data\\DietaryGuidelinesforNINwebsite.pdf', 'total_pages': 139, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject':

In [8]:
len(chunks)

1433

In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


# embed = OllamaEmbeddings(
#     model="nomic-embed-text"
# )



model_name = "sentence-transformers/all-MiniLM-L6-v2"  # or "all-mpnet-base-v2"

hf_embed = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},       # "cuda" if you have a GPU
    encode_kwargs={"normalize_embeddings": False}
)


d:\ML_DL_Project\NLP\p5\llm_chat_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
db = Chroma.from_documents(
    documents=chunks,
    embedding=hf_embed,
    persist_directory="chroma_dbase" 
)

In [28]:
query = 'vitamins A benifits'
results = db.similarity_search(query,k=1)

In [29]:
for i in results:
    print(i.page_content)

Vitamins 
HRPIPER
Page 4 
 
 
Physiological role of Vit A: 
 
 
Vitamin A in Vision: 
 Helps to maintain the cornea 
 Conversion of light energy into nerve impulses at the retina 
 Rhodopsin is a light-sensitive pigment of the retina that contains a protein 
called opsin. 
 
Vitamin A in Protein Synthesis and Cell Differentiation: 
 
Through cell differentiation, vitamin A allows cells to perform specific functions i.e 
Epithelium of skin and mucus membrane (outside of the body form the skin and on the inside of 
the body form the mucous membrane). 
 
Vitamin A in Reproduction and Growth 
 
 Sperm development in men 
 Normal fetal development in women 
 Growth in children 
 Remodeling of the bone involves osteclasts, osteoblasts, and lysosomes. 
 
 
Vitamin A in Immunity 
 
Vita A has been shown to induce production of regulatory T cells that help dampen the immune 
response to self and foreign antigen. 
 
Beta-Carotene as an Antioxidant 
 
Beta-carotene helps protect the body f

In [30]:
#  gemma3:1b
from langchain_community.llms import Ollama

llm = Ollama(model="gemma3:1b")
llm

Ollama(model='gemma3:1b')

In [46]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    You are a Nutrition assistant.
    Provide a detailed, helpful answer.
    no more than 150 words.
    <context>
    {context}
    </context>

    Question: {input}
    """
)


from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


retriever = db.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [47]:
result = retrieval_chain.invoke({'input': 'benefits of vitamin C'})

print(result)


{'input': 'benefits of vitamin C', 'context': [Document(metadata={'keywords': '', 'creator': 'Microsoft® PowerPoint® 2013', 'page': 8, 'format': 'PDF 1.5', 'trapped': '', 'title': 'HEALTH BENEFITS OF FRUITS AND VEGETABLES', 'creationDate': "D:20170821185257+05'30'", 'creationdate': '2017-08-21T18:52:57+05:30', 'modDate': "D:20170821185257+05'30'", 'subject': '', 'file_path': 'pdf_data\\Health benifit of fruits & vegetable.pdf', 'total_pages': 24, 'producer': 'Microsoft® PowerPoint® 2013', 'source': 'pdf_data\\Health benifit of fruits & vegetable.pdf', 'moddate': '2017-08-21T18:52:57+05:30', 'author': 'all'}, page_content='Health benefits and nutrient contribution of some of\nthe fruits and vegetables.\nName of the\nfruit\nNutrient\ncontribution\nHealth benefits\nAmla\nGood\nsource\nof\nvitamin C\n\uf0d8Help as strong antioxidant\n\uf0d8Helps in iron absorption\nApple\nHigh\nin\nsoluble\nfibre and pectin\nReduce blood cholesterol (LDL)\n\uf0d8Counters constipation and diarrhoea\n\uf0d8T

In [48]:
print(result['answer'])

Okay, here's a detailed answer about the benefits of Vitamin C, based on the provided text:

**Vitamin C – A Powerful Nutrient with Numerous Benefits**

Vitamin C is an essential nutrient that plays a vital role in numerous bodily functions. Here’s a breakdown of its key benefits, as highlighted in the text:

*   **Antioxidant Properties:** Vitamin C is a powerful antioxidant, helping protect your cells from damage caused by free radicals.
*   **Iron Absorption:** It helps in the absorption of iron from the gut, which is crucial for carrying oxygen throughout your body.
*   **Wound Healing:** Vitamin C aids in wound repair by supporting cell growth and strengthening the body's immune response.
*   **Bone and Teeth Health:** It’s essential for maintaining strong bones and teeth, and contributes to overall bone health.
*   **Blood Sugar Regulation:** Vitamin C helps to stabilize blood sugar levels, which is beneficial for preventing diabetes.
*   **Vision Improvement:** Vitamin C contrib

In [42]:
def chat_gradio(query):
    result = result = retrieval_chain.invoke({'input': query})

    return result['answer']


In [81]:
import gradio as gr 


with gr.Blocks(theme=gr.themes.Glass(primary_hue="zinc",secondary_hue='rose',neutral_hue='zinc')) as demo:
    gr.Markdown("## Chat Nutrition")

    with gr.Row():
        user_query = gr.Textbox(
            label="Search for Nutrition",
            placeholder="e.g. vitamins, minerals",
            scale=3
        )

        submit_button = gr.Button(value="Submit")


    gr.Markdown("## Info of Nutrition")


    out = gr.Markdown(
        
        value="Answers will appear here.",
        render=True,

    )


    submit_button.click(
        fn=chat_gradio,
        inputs=[user_query],
        outputs=[out]
    )


In [82]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.
